<a href="https://colab.research.google.com/github/jiyoungemma/User-Log-data-analytics/blob/main/recommend_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 라이브러리 & 데이터 불러오기

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import datetime
%matplotlib inline
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [3]:
RFM_without_view = pd.read_csv("/content/drive/MyDrive/RFM_without_view.csv")

In [4]:
RFM_without_view.drop(columns=['field1', 'event_time','user_id.1','category_id', 'Recency','Monetary','Recency_score', 'Frequency_score', 'Monetary_score',
       'RFM_seg', 'RFM_score'], inplace=True)

In [ ]:
RFM_without_view.head()

,event_type,category_code,brand,price,user_id,user_session,main_category,subcategory,Frequency
0,purchase,electronics.smartphone,samsung,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564,electronics,smartphone,33
1,purchase,electronics.smartphone,apple,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68,electronics,smartphone,9
2,purchase,NaN,xiaomi,29.51,514591159,0e5dfc4b-2a55-43e6-8c05-97e1f07fbb56,NaN,NaN,1
3,purchase,furniture.bathroom.toilet,santeri,54.42,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f,furniture,toilet,4
4,purchase,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553,electronics,headphone,1


In [ ]:
RFM_without_view.loc[RFM_without_view['Frequency'] == 1]

,event_type,product_id,category_code,brand,price,user_id,user_session,main_category,subcategory,Frequency,RFM_label
2,purchase,5100816,NaN,xiaomi,29.51,514591159,0e5dfc4b-2a55-43e6-8c05-97e1f07fbb56,NaN,NaN,1,segment5
4,purchase,4804055,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553,electronics,headphone,1,segment5
8,purchase,6200687,appliances.environment.air_heater,oasis,28.03,548691404,b67cdbcb-b073-4271-b365-803c6fce53b0,appliances,air_heater,1,segment5
12,cart,1004833,electronics.smartphone,samsung,174.76,536415846,685b5b42-f597-4a69-ab4c-ef96a30bc454,electronics,smartphone,1,segment5
20,purchase,4804295,electronics.audio.headphone,xiaomi,23.13,540557704,0c990471-9407-4292-a6df-0cbc2f722a40,electronics,headphone,1,segment5
...,...,...,...,...,...,...,...,...,...,...,...
1382811,cart,5100816,NaN,xiaomi,29.58,514905226,6f25d2de-bb1c-4e68-b848-f03372fb6e7d,NaN,NaN,1,segment4
1382814,cart,1004250,electronics.smartphone,apple,787.50,514503441,d2169b3a-422b-4887-9149-ddf35e68e537,electronics,smartphone,1,segment2
1382816,purchase,1004250,electronics.smartphone,apple,787.50,514503441,d2169b3a-422b-4887-9149-ddf35e68e537,electronics,smartphone,1,segment2
1382818,purchase,5100816,NaN,xiaomi,29.58,514905226,6f25d2de-bb1c-4e68-b848-f03372fb6e7d,NaN,NaN,1,segment4


In [ ]:
RFM_without_view.shape

(1382831, 11)

In [ ]:
(530997 / 1382831) * 100

38.39926932502959

### computers dateframe

In [43]:
com_once = RFM_without_view.loc[(RFM_without_view['Frequency'] < 2) & (RFM_without_view['event_type'] == 'purchase') & (RFM_without_view['main_category'] == 'computers')]

In [45]:
com_once.drop(columns = ['event_type','product_id', 'main_category', 'subcategory','Frequency','RFM_label'], inplace=True)
com_once = com_once.reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [46]:
com_once.head()

,category_code,brand,price,user_id,user_session
0,computers.peripherals.printer,epson,116.73,513489079,63de1120-9d48-45d0-b024-500d1b95d71f
1,computers.components.hdd,kingston,66.67,523117953,629a0485-36c9-4a80-9f02-4a96382c3059
2,computers.notebook,acer,532.81,549790258,56272bb8-9f35-4412-9f15-624c8ecda0f1
3,computers.notebook,hp,285.51,555467053,8d2582ff-8031-4b06-819b-a653eb1cb832
4,computers.notebook,hp,285.51,521162064,5622db13-66d1-4167-8bfa-b4f184aed29a


In [47]:
com_once.isna().sum()

category_code      0
brand            228
price              0
user_id            0
user_session       0
dtype: int64

In [48]:
com_once.dropna(inplace=True)

### computers 제품간 추천

In [49]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_tfidf = tfidf.fit_transform(com_once['category_code'])

In [31]:
dtm_tfidf_df= pd.DataFrame(dtm_tfidf.todense(), columns=tfidf.get_feature_names_out())
dtm_tfidf_df

,camera,components,computers,cooler,cpu,desktop,ebooks,hdd,keyboard,memory,monitor,motherboard,mouse,notebook,peripherals,power_supply,printer,videocards
0,0.0,0.00000,0.237006,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.613976,0.0,0.752903,0.0
1,0.0,0.59821,0.184131,0.0,0.0,0.000000,0.0,0.779898,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
2,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
3,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
4,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0
9121,0.0,0.00000,0.301918,0.0,0.0,0.953334,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
9122,0.0,0.00000,0.191205,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.847403,0.000000,0.495327,0.0,0.000000,0.0
9123,0.0,0.00000,0.534484,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.845179,0.000000,0.0,0.000000,0.0


In [50]:
cosine_matrix = cosine_similarity(dtm_tfidf, dtm_tfidf)

In [51]:
np.round(cosine_matrix, 4)

array([[1.    , 0.0436, 0.1267, ..., 0.3494, 0.1267, 0.1267],
       [0.0436, 1.    , 0.0984, ..., 0.0352, 0.0984, 0.0984],
       [0.1267, 0.0984, 1.    , ..., 0.1022, 1.    , 1.    ],
       ...,
       [0.3494, 0.0352, 0.1022, ..., 1.    , 0.1022, 0.1022],
       [0.1267, 0.0984, 1.    , ..., 0.1022, 1.    , 1.    ],
       [0.1267, 0.0984, 1.    , ..., 0.1022, 1.    , 1.    ]])

In [52]:
user = {}
for i, c in enumerate(com_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [53]:
idx = user_id[523117953]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(29, 1.0), (46, 1.0), (67, 1.0), (84, 1.0), (92, 1.0)]

In [35]:
com_once.loc[com_once['user_id'] == 523117953]

,product_id,category_code,brand,price,user_id,user_session,RFM_label
1,6800659,computers.components.hdd,kingston,66.67,523117953,629a0485-36c9-4a80-9f02-4a96382c3059,segment5


In [45]:
com_once.iloc[[92]]

,product_id,category_code,brand,price,user_id,subcategory,RFM_label
96,6801059,computers.components.hdd,seagate,66.8,550318201,hdd,segment5


In [39]:
com_once.iloc[[29]]

,product_id,category_code,brand,price,user_id,subcategory,RFM_label
29,6801082,computers.components.hdd,apacer,24.2,517465372,hdd,segment5


In [40]:
com_once.iloc[[46]]

,product_id,category_code,brand,price,user_id,subcategory,RFM_label
48,6800658,computers.components.hdd,kingston,38.32,517010638,hdd,segment5


In [41]:
com_once.iloc[[67]]

,product_id,category_code,brand,price,user_id,subcategory,RFM_label
71,6800119,computers.components.hdd,toshiba,39.02,531895455,hdd,segment5


In [42]:
com_once.iloc[[84]]

,product_id,category_code,brand,price,user_id,subcategory,RFM_label
88,6801076,computers.components.hdd,apacer,34.75,518667476,hdd,segment5


### computers 유저간 추천

In [54]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_tfidf_user = tfidf.fit_transform(com_once['user_session'])

In [ ]:
dtm_tfidf_user_df= pd.DataFrame(dtm_tfidf_user.todense(), columns=tfidf.get_feature_names_out())
dtm_tfidf_user_df

,0002,00025dab,0002c5ea,0005,0006bf97f732,0007ab79ced4,0011,0012d784,001312a0142e,00153b60a6e5,...,ffeb,ffed,ffee,fff6,fff60634,fff661e40162,fff9,fffa,fffac901e3c9,fffc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
cosine_matrix = cosine_similarity(dtm_tfidf_user, dtm_tfidf_user)

In [56]:
np.round(cosine_matrix, 4)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [57]:
user = {}
for i, c in enumerate(com_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [58]:
idx_category = user_id[523117953]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx_category]) if i != idx_category] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(3212, 0.191554480946969),
 (536, 0.17333571871089953),
 (377, 0.17227939898059863),
 (4214, 0.17227939898059863),
 (0, 0.0)]

In [59]:
com_once.loc[com_once['user_session'] == '629a0485-36c9-4a80-9f02-4a96382c3059']

,category_code,brand,price,user_id,user_session
1,computers.components.hdd,kingston,66.67,523117953,629a0485-36c9-4a80-9f02-4a96382c3059


In [60]:
com_once.iloc[[3212]]

,category_code,brand,price,user_id,user_session
3284,computers.peripherals.printer,epson,244.28,527550767,efd66ffb-9f02-4313-979f-dba38f8c3464


In [61]:
com_once.iloc[[536]]

,category_code,brand,price,user_id,user_session
549,computers.notebook,hp,320.35,551478118,9c664cb4-9311-4a80-bd17-fca008e49c98


In [62]:
com_once.iloc[[377]]

,category_code,brand,price,user_id,user_session
388,computers.components.hdd,transcend,45.05,548837228,165a2320-9850-4a80-845b-1b1fcbc1143d


In [63]:
com_once.iloc[[4214]]

,category_code,brand,price,user_id,user_session
4309,computers.notebook,asus,540.53,559683462,70d3de0c-ce16-4a80-b6b0-e69b3393af9a


### appliances 제품간 추천

In [5]:
appli_once = RFM_without_view.loc[(RFM_without_view['Frequency'] < 2) & (RFM_without_view['event_type'] == 'purchase') & (RFM_without_view['main_category'] == 'appliances')]

In [6]:
appli_once.drop(columns=['event_type','product_id','main_category','subcategory','Frequency', 'RFM_label'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
appli_once = appli_once.reset_index(drop=True)

In [8]:
appli_once.head()

,category_code,brand,price,user_id,user_session
0,appliances.environment.air_heater,oasis,28.03,548691404,b67cdbcb-b073-4271-b365-803c6fce53b0
1,appliances.kitchen.meat_grinder,philips,120.95,553819262,94b897fc-80c6-492c-a456-707398c7cc8e
2,appliances.kitchen.washer,lg,308.65,513419646,5e227e58-6117-419c-b45d-ac54be44716d
3,appliances.kitchen.refrigerators,NaN,204.33,516426855,788ec8fe-f198-4484-b911-ba5841c55c68
4,appliances.personal.hair_cutter,moser,47.88,514018346,35323009-6410-4412-930e-e7855384a5da


In [65]:
appli_once.shape

(31936, 5)

In [9]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_appli = tfidf.fit_transform(appli_once['category_code'])

In [67]:
dtm_appli_df= pd.DataFrame(dtm_appli.todense(), columns=tfidf.get_feature_names_out())
dtm_appli_df

,air_conditioner,air_heater,appliances,blender,coffee_grinder,coffee_machine,dishwasher,environment,fan,grill,...,oven,personal,refrigerators,scales,sewing_machine,steam_cooker,toster,vacuum,washer,water_heater
0,0.0,0.862929,0.196043,0.0,0.0,0.0,0.0,0.465747,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
1,0.0,0.000000,0.210327,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
2,0.0,0.000000,0.317425,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.82639,0.0
3,0.0,0.000000,0.293136,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.85416,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
4,0.0,0.000000,0.145957,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.631627,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31931,0.0,0.000000,0.185239,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
31932,0.0,0.000000,0.317425,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.82639,0.0
31933,0.0,0.000000,0.226555,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0
31934,0.0,0.000000,0.212498,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.977161,0.0,0.0,0.0,0.00000,0.0


In [10]:
cosine_matrix = cosine_similarity(dtm_appli, dtm_appli)

In [11]:
np.round(cosine_matrix, 4)

array([[1.    , 0.0412, 0.0622, ..., 0.0444, 0.0417, 0.0622],
       [0.0412, 1.    , 0.2101, ..., 0.15  , 0.0447, 0.2101],
       [0.0622, 0.2101, 1.    , ..., 0.2263, 0.0675, 1.    ],
       ...,
       [0.0444, 0.15  , 0.2263, ..., 1.    , 0.0481, 0.2263],
       [0.0417, 0.0447, 0.0675, ..., 0.0481, 1.    , 0.0675],
       [0.0622, 0.2101, 1.    , ..., 0.2263, 0.0675, 1.    ]])

In [12]:
user = {}
for i, c in enumerate(appli_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [13]:
idx = user_id[547720935]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(5, 1.0), (124, 1.0), (182, 1.0), (184, 1.0), (220, 1.0)]

In [14]:
appli_once.loc[appli_once['user_id'] == 547720935]

,category_code,brand,price,user_id,user_session
1018,appliances.kitchen.hood,hansa,102.68,547720935,4ff62316-20c5-4edf-a9c5-0e9bca818a18


In [15]:
appli_once.iloc[[5]]

,category_code,brand,price,user_id,user_session
5,appliances.kitchen.hood,artel,61.24,518431897,d244ccce-4dbb-4ae6-af48-98eb36069e19


In [73]:
appli_once.iloc[[124]]

,product_id,category_code,brand,price,user_id
124,2400103,appliances.kitchen.hood,turboair,51.46,540583065


In [72]:
appli_once.iloc[[182]]

,product_id,category_code,brand,price,user_id
182,2400930,appliances.kitchen.hood,hansa,205.67,555357759


In [71]:
appli_once.iloc[[184]]

,product_id,category_code,brand,price,user_id
184,2400953,appliances.kitchen.hood,gorenje,154.18,550314548


In [70]:
appli_once.iloc[[220]]

,product_id,category_code,brand,price,user_id
220,2402570,appliances.kitchen.hood,bosch,102.94,555514854


### appliances 유저간 추천

In [16]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_appli_user = tfidf.fit_transform(appli_once['user_session'])

In [ ]:
dtm_appli_user_df= pd.DataFrame(dtm_appli_user.todense(), columns=tfidf.get_feature_names_out())
dtm_appli_user_df

In [17]:
cosine_matrix = cosine_similarity(dtm_appli_user, dtm_appli_user)

In [18]:
np.round(cosine_matrix, 4)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [19]:
user = {}
for i, c in enumerate(appli_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [21]:
idx = user_id[547720935]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(23095, 0.2096893344877848),
 (12106, 0.18078621866047573),
 (5232, 0.18013754090842443),
 (5233, 0.18013754090842443),
 (5983, 0.17730991497607076)]

In [20]:
appli_once.loc[appli_once['user_id'] == 547720935]

,category_code,brand,price,user_id,user_session
1018,appliances.kitchen.hood,hansa,102.68,547720935,4ff62316-20c5-4edf-a9c5-0e9bca818a18


In [17]:
appli_once.iloc[[23095]]

,product_id,category_code,brand,price,user_id,user_session
23095,3901044,appliances.environment.water_heater,aqua,51.22,528137982,b345ec1c-20c5-42fe-836c-b901bbfd1668


In [18]:
appli_once.iloc[[12106]]

,product_id,category_code,brand,price,user_id,user_session
12106,3200157,appliances.kitchen.meat_grinder,bosch,149.27,519289318,1937dc12-42e1-4f79-a9c5-7f4ea92c3011


In [19]:
appli_once.iloc[[5232]]

,product_id,category_code,brand,price,user_id,user_session
5232,3800548,appliances.iron,vitek,28.29,514528702,5c59ea44-2773-4969-a9c5-1217a6cb6d36


In [20]:
appli_once.iloc[[5233]]

,product_id,category_code,brand,price,user_id,user_session
5233,3800548,appliances.iron,vitek,28.29,514528702,5c59ea44-2773-4969-a9c5-1217a6cb6d36


In [21]:
appli_once.iloc[[5983]]

,product_id,category_code,brand,price,user_id,user_session
5983,2402401,appliances.kitchen.hood,gorenje,141.29,513669164,cdd6e5f3-198f-4a27-a9c5-f509eb9edd0b


### electronics 제품간 추천
- segment5에 대한 추천

In [5]:
elec_once = RFM_without_view.loc[(RFM_without_view['Frequency'] < 2) & (RFM_without_view['event_type'] == 'purchase') & (RFM_without_view['main_category'] == 'electronics') & (RFM_without_view['RFM_label'] == 'segment5')]

In [6]:
elec_once['RFM_label'].value_counts()

segment4    64996
segment3    44152
segment5    22159
segment2     7114
Name: RFM_label, dtype: int64

In [6]:
elec_once.drop(columns=['event_type','product_id', 'main_category','subcategory','Frequency','RFM_label'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
elec_once.reset_index(drop=True, inplace=True)

In [8]:
elec_once.head()

,category_code,brand,price,user_id,user_session
0,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553
1,electronics.audio.headphone,xiaomi,23.13,540557704,0c990471-9407-4292-a6df-0cbc2f722a40
2,electronics.smartphone,huawei,111.82,512812487,d3ef047f-9476-4b8d-a126-b483c55de653
3,electronics.smartphone,huawei,177.20,518962372,718c8069-2782-4585-acad-6747f6f2f79d
4,electronics.smartphone,xiaomi,185.71,552439521,80ccabf1-39b6-421f-b452-4bf9e37805d6


In [9]:
elec_once.shape

(22159, 5)

In [10]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_elec = tfidf.fit_transform(elec_once['category_code'])

In [11]:
cosine_matrix = cosine_similarity(dtm_elec, dtm_elec)

In [12]:
np.round(cosine_matrix, 4)

array([[1.    , 1.    , 0.1473, ..., 0.1473, 0.1473, 0.0675],
       [1.    , 1.    , 0.1473, ..., 0.1473, 0.1473, 0.0675],
       [0.1473, 0.1473, 1.    , ..., 1.    , 1.    , 0.1474],
       ...,
       [0.1473, 0.1473, 1.    , ..., 1.    , 1.    , 0.1474],
       [0.1473, 0.1473, 1.    , ..., 1.    , 1.    , 0.1474],
       [0.0675, 0.0675, 0.1474, ..., 0.1474, 0.1474, 1.    ]])

In [13]:
user = {}
for i, c in enumerate(elec_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [15]:
idx = user_id[540557704]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(0, 1.0), (31, 1.0), (37, 1.0), (57, 1.0), (74, 1.0)]

In [14]:
elec_once.loc[elec_once['user_id'] == 540557704]

,category_code,brand,price,user_id,user_session
1,electronics.audio.headphone,xiaomi,23.13,540557704,0c990471-9407-4292-a6df-0cbc2f722a40


In [16]:
elec_once.iloc[[0]]

,category_code,brand,price,user_id,user_session
0,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553


In [17]:
elec_once.iloc[[31]]

,category_code,brand,price,user_id,user_session
31,electronics.audio.headphone,apple,161.98,543650701,4c9b3b0d-9511-481a-ba0b-b5350a7cf0b4


In [18]:
elec_once.iloc[[37]]

,category_code,brand,price,user_id,user_session
37,electronics.audio.headphone,marshall,89.83,517163214,af635016-1f2d-4b33-830a-09743991c812


In [19]:
elec_once.iloc[[57]]

,category_code,brand,price,user_id,user_session
57,electronics.audio.headphone,xiaomi,22.39,514536402,4652dea0-b6a0-4dd7-8a0d-8c1323e4d461


In [20]:
elec_once.iloc[[74]]

,category_code,brand,price,user_id,user_session
74,electronics.audio.headphone,apple,188.73,539281255,0615b21d-900c-4df2-8409-b8348011b168


### electronics 유저간 추천
- segment5 에 대한 추천

In [21]:
tfidf = TfidfVectorizer()

# Fit 후 dtm을 만듭니다.(문서, 단어마다 tf-idf 값을 계산합니다)
dtm_elec_user = tfidf.fit_transform(elec_once['user_session'])

In [22]:
cosine_matrix = cosine_similarity(dtm_elec_user, dtm_elec_user)

In [23]:
np.round(cosine_matrix, 4)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [26]:
user = {}
for i, c in enumerate(elec_once['user_id']): 
  user[i] = c

user_id = {}
for i, c in user.items():
  user_id[c] = i

In [35]:
idx = user_id[540557704]
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 유저들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:5] # 상위 5개의 인덱스와 유사도를 추출 

[(8152, 0.19936300100317822),
 (21390, 0.19603531121655107),
 (21421, 0.19603531121655107),
 (18505, 0.16982092407133242),
 (1389, 0.16907715071482754)]

In [34]:
elec_once.loc[elec_once['user_id'] == 540557704]

,category_code,brand,price,user_id,user_session
1,electronics.audio.headphone,xiaomi,23.13,540557704,0c990471-9407-4292-a6df-0cbc2f722a40


In [36]:
elec_once.iloc[[8152]]

,category_code,brand,price,user_id,user_session
8152,electronics.smartphone,huawei,109.36,556943187,58f8c518-29f8-4e00-a6df-af6682f173b6


In [37]:
elec_once.iloc[[21390]]

,category_code,brand,price,user_id,user_session
21390,electronics.audio.headphone,xiaomi,6.18,536671878,f9cec814-9407-4921-9e24-6cad47f47e9a


In [38]:
elec_once.iloc[[21421]]

,category_code,brand,price,user_id,user_session
21421,electronics.audio.headphone,xiaomi,5.79,536671878,f9cec814-9407-4921-9e24-6cad47f47e9a


In [39]:
elec_once.iloc[[18505]]

,category_code,brand,price,user_id,user_session
18505,electronics.audio.headphone,apple,161.87,534776631,513d38f5-ea73-4292-8fcb-eaada6ab10e9


In [40]:
elec_once.iloc[[1389]]

,category_code,brand,price,user_id,user_session
1389,electronics.smartphone,xiaomi,100.12,527259774,548ae7ce-068c-4292-938d-67c4fc2e4294
